### Deliverables:

#### 1. Data Preparation - NLP 

- Removing special characters using regex

-- Importing Modules

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
import re
import spacy

nlp=spacy.load("en_core_web_lg")

In [2]:
df = pd.read_excel("D:/NLP/data/bank_a_sentiment_analysis_translated_inputdata.xlsx")
print(df.info(),"Shape:",df.shape)
df = df[["translated_review","actual_sentiment"]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3486 entries, 0 to 3485
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   review_id          3486 non-null   float64
 1   translated_review  3481 non-null   object 
 2   actual_sentiment   3482 non-null   object 
dtypes: float64(1), object(2)
memory usage: 81.8+ KB
None Shape: (3486, 3)


#### Q1. Provide a report for data sufficiency, sparsity, and anomalies in each data source

In [3]:
df.describe(include="all")

,translated_review,actual_sentiment
count,3481,3482
unique,3306,3
top,because i am fine,Positive
freq,11,1834


In [4]:
# Sparsity
sparsity=(df.to_numpy()==0).mean()
print("Data Sparsity of our dataframe is:",sparsity)

Data Sparsity of our dataframe is: 0.0


#### Q2. Identify and handle empty/ blank/ NA reviews, blanks in the actual sentiment column, very long or short review texts

In [5]:
# Since we cannot impute text data. Therefore, removing empty/blank/NA values from dataset.
df=df.dropna().reset_index()
df=df.drop("index",axis=1)

##### Q3: Identify the special characters to keep and remove from the text

In [6]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)

    return "".join(cleantext)

df['preprocessed_text']=df['translated_review'].map(lambda s:preprocess(s)) 

#### Q4: Perform the required text cleaning operations to suit the model (note that some downstream sentiment models don’t require text cleaning)

In [7]:
df['preprocessed_text'] = df['preprocessed_text'].str.replace('[^\w\s]','')

C:\Users\SAURAB~1.MIS\AppData\Local\Temp/ipykernel_30404/1365977855.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preprocessed_text'] = df['preprocessed_text'].str.replace('[^\w\s]','')


In [8]:
# StopWords Removal
from nltk.corpus import stopwords
# stopwords = en.Defaults.stop_words
stop = stopwords.words('english')
not_stop=["no","not"]
for i in not_stop:
    stop.remove(i)
df['clean_text'] = df['preprocessed_text'].apply(lambda x: " ".join(i for i in x.split() if i not in stop))
# Stemming
df["cleaned_text"]=df["clean_text"].apply(lambda x: " ".join(token.lemma_ for token in nlp(x) if not (token.like_num or token.is_punct or token.is_space or len(token)==1)))
df=df[["cleaned_text","actual_sentiment"]]
df

,cleaned_text,actual_sentiment
0,branch san marco evangelista employee include ...,Positive
1,commission withdraw money counter theft,Negative
2,f24 app thank,Neutral
3,low commission,Neutral
4,promote support small business territory,Positive
...,...,...
3472,availability rapidity,Neutral
3473,serious reliable comfort payment etc,Neutral
3474,norm,Neutral
3475,poor interest customer,Negative


### 2. Exploratory Data Analysis (EDA)

#### Q1.

In [9]:
# !pip install pytorch-pretrained-bert